## Importing necessary libraries

In [1]:
%pwd

'/Users/Nicole/github/heart-failure-project'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
np.random.seed(130298) 

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer 
from fancyimpute import IterativeImputer, KNN

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, f1_score, recall_score, precision_score, roc_curve, mean_squared_error

In [5]:
from sklearn.model_selection import GridSearchCV

In [6]:
def compute_metrics(Y, final_pred):
    fpr, tpr, _ = roc_curve(Y,final_pred)
    AUC = auc(fpr, tpr)
    f1 = f1_score(Y,final_pred)
    rec = recall_score(Y,final_pred)
    prec = precision_score(Y,final_pred)
    print(f'AUC: {AUC}, F1: {f1}, Recall: {rec}, Precision: {prec}')
    return [AUC, f1, rec, prec]

## Datasets definition

In [7]:
outcome_name = 're.admission.within.6.months'

# take the training set
X_train = pd.read_csv('train_data_drugs.csv')
X_train.set_index('inpatient.number', inplace = True)

# separate the outcome
Y_train = X_train[outcome_name].copy()
X_train.drop(columns = outcome_name, inplace = True)

# take the test set
X_test = pd.read_csv('test_data_drugs.csv')
X_test.set_index('inpatient.number', inplace = True)

# separate the outcome
Y_test = X_test[outcome_name].copy()
X_test.drop(columns = outcome_name, inplace = True)


In [8]:
print(f'Size of traing set: {X_train.shape} and of test set: {X_test.shape}')

Size of traing set: (1567, 81) and of test set: (397, 81)


In [9]:
# Log transformations
var_to_log = ['creatinine.enzymatic.method', 'urea', 'glomerular.filtration.rate', 
              'cystatin', 'lymphocyte.count', 'neutrophil.count',
              'activated.partial.thromboplastin.time', 'prothrombin.time.ratio',
              'glutamyltranspeptidase','indirect.bilirubin','alkaline.phosphatase',
              'globulin','direct.bilirubin', 'low.density.lipoprotein.cholesterol', 
              'triglyceride']
X_train[var_to_log] = np.log(X_train[var_to_log])
X_test[var_to_log] = np.log(X_test[var_to_log])

In [10]:
# take the lists of variables by type

cat_columns = ['DestinationDischarge','admission.ward','admission.way','discharge.department',
                       'type.of.heart.failure', 'NYHA.cardiac.function.classification', 'Killip.grade',
                       'consciousness', 'ageCat']

ordinal_columns = ['CCI.score', 'eye.opening','verbal.response', 'movement', 'GCS']

not_continuous = cat_columns.copy()

binary_columns = ['gender', 'diabetes', 'moderate.to.severe.chronic.kidney.disease',
                  'return.to.emergency.department.within.6.months', 'diuretics',
                  'hypertension', 'heart_failure', 'angina_etal', 'cholesterol']

not_continuous.extend(binary_columns)
not_continuous.extend(ordinal_columns)

In [11]:
cont_columns = [col_name for col_name in X_train.columns if col_name not in not_continuous]

In [12]:
# for each category we add its specification, needed to retreive columns after OneHotEcoding
#for cat in cat_columns:
#    X_train[cat] = cat + '_' + X_train[cat]
#    X_test[cat] = cat + '_' + X_test[cat]

## Pipeline definition

In [13]:
#def log_transf_stand(x):
#    "Transform to log and standardize"
#    x = np.log(x)
#    return (x - x.mean())/x.std()

#log_transformer = FunctionTransformer(log_transf_stand)

In [14]:
#full_pipeline = ColumnTransformer([
#        ("log", log_transformer, var_to_log),
#        ("num", StandardScaler(), list(set(cont_columns) - set(var_to_log))),     
#        ("cat", OneHotEncoder(), cat_columns)],
#       remainder = 'passthrough')

In [15]:
full_pipeline = ColumnTransformer([
                ("num", StandardScaler(),cont_columns),
                ("cat", OneHotEncoder(), cat_columns)],
                remainder = 'passthrough') # do not modify columns not listed

X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.transform(X_test) 

In [16]:
print(f'Size of traing set: {X_train_prepared.shape} and of test set: {X_test_prepared.shape}')

Size of traing set: (1567, 107) and of test set: (397, 107)


In [74]:
column_prepared = list(full_pipeline.get_feature_names_out(input_features=X_train.columns))

In [17]:
# imputation cannot be done inside the Pipeline because the method 'transform' is missing

knn_imputer = KNN()
X_train_prepared = knn_imputer.fit_transform(X_train_prepared)
X_test_prepared = knn_imputer.fit_transform(X_test_prepared)

Imputing row 1/1567 with 0 missing, elapsed time: 1.254
Imputing row 101/1567 with 2 missing, elapsed time: 1.260
Imputing row 201/1567 with 3 missing, elapsed time: 1.266
Imputing row 301/1567 with 0 missing, elapsed time: 1.273
Imputing row 401/1567 with 16 missing, elapsed time: 1.278
Imputing row 501/1567 with 4 missing, elapsed time: 1.285
Imputing row 601/1567 with 4 missing, elapsed time: 1.291
Imputing row 701/1567 with 5 missing, elapsed time: 1.296
Imputing row 801/1567 with 1 missing, elapsed time: 1.303
Imputing row 901/1567 with 0 missing, elapsed time: 1.308
Imputing row 1001/1567 with 1 missing, elapsed time: 1.315
Imputing row 1101/1567 with 1 missing, elapsed time: 1.321
Imputing row 1201/1567 with 4 missing, elapsed time: 1.327
Imputing row 1301/1567 with 1 missing, elapsed time: 1.334
Imputing row 1401/1567 with 4 missing, elapsed time: 1.340
Imputing row 1501/1567 with 0 missing, elapsed time: 1.346
Imputing row 1/397 with 9 missing, elapsed time: 0.084
Imputing row

In [20]:
X_train_prepared=pd.DataFrame(X_train_prepared, columns=column_prepared, index=X_train.index)
X_train_prepared.head()

,num__body.temperature,num__pulse,num__respiration,num__systolic.blood.pressure,num__diastolic.blood.pressure,num__weight,num__BMI,num__fio2,num__left.ventricular.end.diastolic.diameter.LV,num__creatinine.enzymatic.method,...,remainder__CCI.score,remainder__eye.opening,remainder__verbal.response,remainder__movement,remainder__GCS,remainder__diuretics,remainder__hypertension,remainder__heart_failure,remainder__angina_etal,remainder__cholesterol
inpatient.number,,,,,,,,,,,,,,,,,,,,,
746794,1.556790,0.650323,-2.421625,1.652486,0.650813,-0.717652,0.066535,0.082293,-0.933569,-0.722531,...,1.0,4.0,5.0,6.0,15.0,1.0,1.0,0.0,1.0,0.0
830900,0.194327,-0.486118,-0.648407,0.918719,0.511105,3.537261,0.863395,0.082293,0.462482,-0.774648,...,1.0,4.0,5.0,6.0,15.0,1.0,0.0,1.0,1.0,1.0
730511,4.962947,0.508268,-0.057334,-0.059637,0.231688,-0.489843,-1.433437,0.082293,0.741692,-0.087419,...,3.0,4.0,5.0,6.0,15.0,1.0,1.0,1.0,1.0,0.0
790988,1.329713,0.129454,-0.648407,-0.956464,-1.584523,-1.001313,-0.213904,0.082293,-0.188636,3.288521,...,2.0,4.0,5.0,6.0,15.0,1.0,0.0,0.0,0.0,1.0
779438,0.194327,0.981785,-0.648407,0.755659,0.930230,0.511545,0.296504,-0.847097,-0.002868,0.835269,...,2.0,4.0,5.0,6.0,15.0,1.0,1.0,1.0,1.0,1.0


In [21]:
X_test_prepared=pd.DataFrame(X_test_prepared, columns=column_prepared, index=X_test.index)
X_test_prepared.head()

,num__body.temperature,num__pulse,num__respiration,num__systolic.blood.pressure,num__diastolic.blood.pressure,num__weight,num__BMI,num__fio2,num__left.ventricular.end.diastolic.diameter.LV,num__creatinine.enzymatic.method,...,remainder__CCI.score,remainder__eye.opening,remainder__verbal.response,remainder__movement,remainder__GCS,remainder__diuretics,remainder__hypertension,remainder__heart_failure,remainder__angina_etal,remainder__cholesterol
inpatient.number,,,,,,,,,,,,,,,,,,,,,
770068,0.194327,-0.249359,-0.648407,-0.263462,0.231688,2.118957,0.744300,0.082293,0.149976,0.312704,...,1.0,4.0,5.0,6.0,15.0,1.0,1.0,1.0,1.0,0.0
860037,-0.941059,0.602971,-0.648407,0.755659,0.720667,2.024403,0.259282,0.082293,0.369412,-0.491654,...,1.0,4.0,5.0,6.0,15.0,1.0,1.0,1.0,1.0,1.0
782110,0.194327,-0.249359,-0.648407,-0.874934,0.231688,0.511545,0.046743,-2.705875,0.595091,0.393034,...,3.0,4.0,5.0,6.0,15.0,1.0,1.0,1.0,1.0,0.0
742279,-0.032750,-1.385801,-0.648407,-1.078759,-0.816126,-1.190420,-0.271977,0.082293,-0.301401,-1.061928,...,1.0,4.0,5.0,6.0,15.0,1.0,1.0,0.0,1.0,1.0
734121,0.194327,-1.243745,-0.057334,0.266481,0.790522,-0.717652,-0.353568,0.082293,-0.468219,0.717118,...,3.0,4.0,5.0,6.0,15.0,1.0,1.0,0.0,1.0,0.0


In [22]:
X_train_prepared.drop(columns = "cat__DestinationDischarge_nan",inplace=True)
X_test_prepared.drop(columns = "cat__DestinationDischarge_nan",inplace=True)
column_prepared.remove("cat__DestinationDischarge_nan")

In [23]:
print(f'Size of traing set: {X_train_prepared.shape} and of test set: {X_test_prepared.shape}')

Size of traing set: (1567, 106) and of test set: (397, 106)


In [ ]:
#for idx,elem in enumerate(features_names):
#    if elem == 'nan':
#        print(idx)

In [ ]:
#X_train_prepared = np.delete(X_train_prepared, idx, axis = 1)
#X_test_prepared = np.delete(X_test_prepared, idx, axis = 1)

# ADJUST THIS!!!

## Logistic Regression with Elasticnet penalty
Since we have many columns, of which we doubt some might still be collinear, we use an Elastic net penalty, which takes into account both the L2 norm and the L1 norm, which induces sparsity.

Moreover, we add the “balanced” mode, which uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).

In [31]:
log_reg_clf = LogisticRegression(penalty = 'elasticnet', solver = 'saga', 
                                 class_weight = 'balanced', l1_ratio = 0.5, 
                                 max_iter = 5000, random_state=42)
# this method takes many iterations to converge
log_reg_clf.fit(X_train_prepared, Y_train)

LogisticRegression(class_weight='balanced', l1_ratio=0.5, max_iter=5000,
                   penalty='elasticnet', random_state=42, solver='saga')

In [32]:
pred = log_reg_clf.predict(X_test_prepared)
final_mse = mean_squared_error(Y_test, pred)
final_rmse = np.sqrt(final_mse)
final_rmse

0.6228236667783622

In [33]:
compute_metrics(Y_test, pred)

AUC: 0.607022607022607, F1: 0.5389221556886228, Recall: 0.5844155844155844, Precision: 0.5


[0.607022607022607, 0.5389221556886228, 0.5844155844155844, 0.5]

In [34]:
from sklearn.feature_selection import SelectFromModel

model = SelectFromModel(log_reg_clf, prefit=True)
X_new = model.transform(X_train_prepared)
X_new.shape

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


(1567, 35)

In [35]:
significant_features=list(model.get_feature_names_out(input_features=column_prepared))

In [ ]:
# TO GET THE NAMES OF THE IMPORTANT FEATURES, WE HAVE TO PASS THE MATRIX WITH THE COLUMN NAMES

In [ ]:
# AFTER THIS, FIT THE RANDOM FOREST ONLY WITH THE GOOD FEATURES!!

In [ ]:
#log_reg_clf = LogisticRegression(penalty = 'elasticnet', solver = 'saga', class_weight = 'balanced', max_iter = 1000, random_state=42)

# C and l1_ratio can be tuned
#'C': np.linspace(0.1,5,100),
#param_grid = {'l1_ratio': np.linspace(0,1,101)}
#grid_search = GridSearchCV(log_reg_clf, param_grid, cv=10,
#                           scoring='neg_mean_squared_error',
#                           return_train_score=True)
#grid_search.fit(X_train_prepared, Y_train)

## Random Forest classifier

In [91]:
param_grid = {'n_estimators': [10, 30, 50, 100],
              'max_features': list(np.arange(10,X_train_prepared.shape[1],10)) + [X_train_prepared.shape[1]]} 

    

forest_clf = RandomForestClassifier(random_state=42)

# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_clf, param_grid, cv=10,
                           scoring='f1_weighted',
                           return_train_score=True)
grid_search.fit(X_train_prepared, Y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_features': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100, 106],
                         'n_estimators': [10, 30, 50, 100]},
             return_train_score=True, scoring='f1_weighted')

In [92]:
final_model = grid_search.best_estimator_
final_model

RandomForestClassifier(max_features=106, random_state=42)

### Fetaure importance

In [93]:
feature_importances = final_model.feature_importances_
selected_features = sorted(list(zip(feature_importances,significant_features)), reverse=True)

In [94]:
final_model.fit(X_train_prepared,Y_train)

RandomForestClassifier(max_features=106, random_state=42)

In [95]:
selected_features_name = [elem[1] for elem in selected_features]

### Predict on test set

In [96]:
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

0.5938390659699312

In [97]:
compute_metrics(Y_test, final_predictions)

AUC: 0.5894527283416172, F1: 0.4214876033057851, Recall: 0.33116883116883117, Precision: 0.5795454545454546


[0.5894527283416172,
 0.4214876033057851,
 0.33116883116883117,
 0.5795454545454546]

In [ ]:
#cat_list_one_hot = full_pipeline.named_transformers_["cat"].categories_

In [ ]:
#cat_features = []
#for elem in cat_list_one_hot:
#    cat_features.extend(elem)
#cat_features

In [ ]:
sorted(zip(feature_importances,features_names), reverse=True)